In [1]:
from profiling_utils import create_remote_backend_from_triplets
from rag_feature_store import SentenceTransformerFeatureStore
from rag_graph_store import NeighborSamplingRAGGraphStore
from rag_loader import RagQueryLoader
from torch_geometric.datasets import UpdatedWebQSPDataset
from torch_geometric.nn.nlp import SentenceTransformer
from torch_geometric.datasets.updated_web_qsp_dataset import preprocess_triplet
from itertools import chain
import torch

/home/zaristei/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = UpdatedWebQSPDataset("small_ds", force_reload=True, limit=10)

Processing...


Loading graph...
Encoding questions...
Retrieving subgraphs...


10it [00:00, 59.31it/s]00:00<?, ?it/s]
100%|██████████| 10/10 [00:01<00:00,  5.89it/s]

Saving subgraphs...



Done!


In [3]:
triplets = list(chain.from_iterable((d['graph'] for d in ds.raw_dataset)))

In [4]:
del ds

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer().to(device)

In [6]:
fs, gs = create_remote_backend_from_triplets(triplets=triplets, node_embedding_model=model, node_method_to_call="encode", path="backend", pre_transform=preprocess_triplet, node_method_kwargs={"batch_size": 256}, graph_db=NeighborSamplingRAGGraphStore, feature_db=SentenceTransformerFeatureStore).load()

In [7]:
query_loader = RagQueryLoader(data=(fs, gs))

In [8]:
query = ["Who is Justin bieber?",
         "How many bachelors are married?"]

In [10]:
subg = query_loader.query(query)

ValueError: only one element tensors can be converted to Python scalars